In [1]:
!pip install llama-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.2/129.2 kB 7.8 MB/s eta 0:00:00


In [19]:
pip install  llama-index-llms-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [20]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel
from typing import List, Dict, Optional
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.huggingface import HuggingFaceLLM

class SmallAgent:
    def __init__(self, model_name: str = "bert-base-uncased"):
        # Инициализация модели и токенизатора
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)

        # Простой классификатор поверх BERT
        self.classifier = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Linear(256, 2)  # Бинарная классификация
        )

        # Настройка для llama-index
        self.llm = HuggingFaceLLM(
            model_name="gpt2",
            tokenizer_name="gpt2",
            device_map="auto"
        )

        # Инициализация индекса (пустой, будет заполнен позже)
        self.index = None

    def embed_text(self, text: str) -> torch.Tensor:
        """Создание эмбеддинга для текста"""
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            outputs = self.model(**inputs)
        return outputs.last_hidden_state.mean(dim=1)  # Усреднение по токенам

    def train_classifier(self, texts: List[str], labels: List[int], epochs: int = 3):
        """Обучение классификатора на текстах"""
        optimizer = torch.optim.Adam(self.classifier.parameters(), lr=1e-4)
        criterion = nn.CrossEntropyLoss()

        for epoch in range(epochs):
            total_loss = 0
            for text, label in zip(texts, labels):
                # Получаем эмбеддинг
                embedding = self.embed_text(text)

                # Прямой проход
                logits = self.classifier(embedding)
                loss = criterion(logits, torch.tensor([label]))

                # Обратный проход
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                total_loss += loss.item()

            print(f"Epoch {epoch+1}, Loss: {total_loss/len(texts):.4f}")

    def predict(self, text: str) -> int:
        """Предсказание класса для текста"""
        embedding = self.embed_text(text)
        with torch.no_grad():
            logits = self.classifier(embedding)
        return torch.argmax(logits).item()

    def build_knowledge_base(self, directory: str):
        """Построение базы знаний из документов в директории"""
        documents = SimpleDirectoryReader(directory).load_data()
        self.index = VectorStoreIndex.from_documents(documents, llm=self.llm)

    def query_knowledge(self, question: str) -> str:
        """Запрос к базе знаний"""
        if self.index is None:
            return "Knowledge base not initialized. Call build_knowledge_base() first."

        query_engine = self.index.as_query_engine()
        response = query_engine.query(question)
        return str(response)

# Пример использования
if __name__ == "__main__":
    agent = SmallAgent()

    # Пример обучения классификатора
    texts = [
        "I love this product!",
        "This is terrible.",
        "Amazing experience!",
        "Worst purchase ever."
    ]
    labels = [1, 0, 1, 0]  # 1 - позитив, 0 - негатив

    print("Training classifier...")
    agent.train_classifier(texts, labels)

    # Тестирование классификатора
    test_text = "This is really good!"
    print(f"Prediction for '{test_text}': {'Positive' if agent.predict(test_text) else 'Negative'}")

    # Пример работы с базой знаний (нужно создать папку 'data' с документами)
    try:
        print("\nBuilding knowledge base...")
        agent.build_knowledge_base("data")

        question = "What is the main topic of the documents?"
        print(f"Answer to '{question}': {agent.query_knowledge(question)}")
    except Exception as e:
        print(f"Knowledge base error: {e}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Training classifier...
Epoch 1, Loss: 0.6732
Epoch 2, Loss: 0.6115
Epoch 3, Loss: 0.5665
Prediction for 'This is really good!': Positive

Building knowledge base...


Knowledge base error: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
